<a href="https://colab.research.google.com/github/shubbham37/Covid19-Confirmed-cases-prediction/blob/master/Covid19%20Prediction%20based%20on%20questionnaire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import lightgbm as lgb
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization

In [ ]:
# view summary of dataset
# view summary of dataset
df = pd.read_csv('/content/drive/My Drive/corona_tested_individuals_ver_006.english.csv')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head()


In [ ]:
df.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication,latitude,longitude
0,2020-04-30,0,0,0,0,0,negative,None,female,Other,15.0000,101.0000
1,2020-04-30,1,0,0,0,0,negative,None,female,Other,36.0000,138.0000
2,2020-04-30,0,1,0,0,0,negative,None,male,Other,1.2833,103.8333
3,2020-04-30,1,0,0,0,0,negative,None,female,Other,28.1667,84.2500
4,2020-04-30,1,0,0,0,0,negative,None,male,Other,2.5000,112.5000


In [ ]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
df['corona_result']= label_encoder.fit_transform(df['corona_result'])
df['age_60_and_above']= label_encoder.fit_transform(df['age_60_and_above'])
df['gender']= label_encoder.fit_transform(df['gender'])
df['test_indication']= label_encoder.fit_transform(df['test_indication'])


df.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication,latitude,longitude
0,2020-04-30,0,0,0,0,0,0,0,1,2,15.0000,101.0000
1,2020-04-30,1,0,0,0,0,0,0,1,2,36.0000,138.0000
2,2020-04-30,0,1,0,0,0,0,0,2,2,1.2833,103.8333
3,2020-04-30,1,0,0,0,0,0,0,1,2,28.1667,84.2500
4,2020-04-30,1,0,0,0,0,0,0,2,2,2.5000,112.5000


In [ ]:
df.drop('test_date',axis='columns', inplace=True)

In [ ]:
# view summary of dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cough                10000 non-null  int64  
 1   fever                10000 non-null  int64  
 2   sore_throat          10000 non-null  int64  
 3   shortness_of_breath  10000 non-null  int64  
 4   head_ache            10000 non-null  int64  
 5   corona_result        10000 non-null  int64  
 6   age_60_and_above     10000 non-null  int64  
 7   gender               10000 non-null  int64  
 8   test_indication      10000 non-null  int64  
 9   latitude             10000 non-null  float64
 10  longitude            10000 non-null  float64
dtypes: float64(2), int64(9)
memory usage: 937.5 KB


In [ ]:
df=df.mask(df.eq('None')).dropna()



In [ ]:
df['cough'] = pd.to_numeric(df['cough'])
df['fever'] = pd.to_numeric(df['fever'])
df['sore_throat'] = pd.to_numeric(df['sore_throat'])
df['head_ache'] = pd.to_numeric(df['head_ache'])
df['shortness_of_breath'] = pd.to_numeric(df['shortness_of_breath'])


In [ ]:
X = df[['cough','fever','sore_throat','shortness_of_breath','head_ache','age_60_and_above','gender','test_indication','latitude','longitude']]
y = df['corona_result']


In [ ]:
X.head()

,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender,test_indication,latitude,longitude
0,0,0,0,0,0,0,1,2,15.0000,101.0000
1,1,0,0,0,0,0,1,2,36.0000,138.0000
2,0,1,0,0,0,0,2,2,1.2833,103.8333
3,1,0,0,0,0,0,1,2,28.1667,84.2500
4,1,0,0,0,0,0,2,2,2.5000,112.5000


In [ ]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: corona_result, dtype: int64

In [ ]:
# split the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(num_leaves= 20, min_data_in_leaf= 4, feature_fraction= 0.2, bagging_fraction= 0.8, bagging_freq= 5, learning_rate= 0.05,verbose=1)
clf.fit(X_train, y_train)

LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.2,
               importance_type='split', learning_rate=0.05, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_data_in_leaf=4,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=20,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0, verbose=1)

In [ ]:
y_pred=clf.predict(X_test)

y_pred[0]


0

In [ ]:
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

LightGBM Model accuracy score: 0.9740


In [ ]:
y_pred_train = clf.predict(X_train)

print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

Training-set accuracy score: 0.9790


In [ ]:
# print the scores on training and test set

print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))

Training set score: 0.9790
Test set score: 0.9740


In [ ]:
# view confusion-matrix
# Print the Confusion Matrix and slice it into four pieces

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0,0])
print('\nTrue Negatives(TN) = ', cm[1,1])
print('\nFalse Positives(FP) = ', cm[0,1])
print('\nFalse Negatives(FN) = ', cm[1,0])

Confusion matrix

 [[2913    0    5]
 [  22    0    0]
 [  51    0    9]]

True Positives(TP) =  2913

True Negatives(TN) =  0

False Positives(FP) =  0

False Negatives(FN) =  22


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2918
           1       0.00      0.00      0.00        22
           2       0.64      0.15      0.24        60

    accuracy                           0.97      3000
   macro avg       0.54      0.38      0.41      3000
weighted avg       0.96      0.97      0.96      3000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
!pip install flask_ngrok

In [ ]:
# flask_ngrok_example.py
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import request, jsonify
from collections import defaultdict
import json

answer_dict = {'YES':1,'NO':0,'Male':1,'Female':0}
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/",methods=['GET', 'POST'])
def hello():
    data = request.json
    features = []
    
    for obj in data:
      if obj.get('question') != 'Address':
        features.append(answer_dict[obj['selected_answer'][0]['answer_choice']])
    answer = clf.predict([features])
    print(answer)
    return json.dumps([{'answer':str(answer[0])}])

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://dbd4578faec3.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[2021-06-10 14:24:12,441] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-35-558956890d50>", line 17, in hello
    for obj in data:
TypeError: 'NoneType' object is n